`_230210-0303_math-problem-randomization-library`

### `Front` and `Back` templates are setup like so:

`Front` template:

```html
<script>
  // Avoid redefining pseudorandom variable.
  if (typeof pseudorandom === 'undefined') { let pseudorandom; }
  if (`{{randomizer-Index:}}`.startsWith('Unknown field:')) {
    // If custom field isn't available, default to zero.
    pseudorandom = 0;
  } else {
    pseudorandom = {{randomizer-Index:}};
  }

  // Run JavaScript snippet from card to interpolate random
  // params for displaying front and back sides.
  {{Params}}

  // Copy raw interpolated Front to div below.
  document.getElementById('parameterized_front').innerHTML = String.raw`{{Front}}`;
  console.log(`pseudorandom: ${pseudorandom}`);
</script>

<div id='parameterized_front'>
  {{Front}}
  <br>
  (Warning: Front not yet defined by card template script.)
</div>
```

`Back` template:

```html

{{FrontSide}}

<hr id=answer>

<script>
  // Random seed and params have  already been setup in Front  template.
  // Copy raw interpolated Back to div below.
  document.getElementById('parameterized_back').innerHTML = String.raw`{{Back}}`;
</script>

<div id='parameterized_back'>
  {{Back}}
  <br>
  (Warning: Back not yet defined by card template script.)
</div>
```

### And Anki card using the above template are setup like so:

Example card `Params` field:

```javascript
// Example:
if (typeof params === 'undefined') { let params, a, b; }
params = [
  [1,2],
  [1,4],
  [2,3],
];
[a, b] = params[pseudorandom % params.length];
```

Variant with full expression:

```javascript
if (typeof params === 'undefined') { let params, a, ex; }
params = [
  [2, String.raw`\left(z^{2} - 2 z + 2\right) e^{z}`],
  [3, String.raw`\left(z^{3} - 3 z^{2} + 6 z - 6\right) e^{z}`],
  [4, String.raw`\left(z^{4} - 4 z^{3} + 12 z^{2} - 24 z + 24\right) e^{z}`],
  [5, String.raw`\left(z^{5} - 5 z^{4} + 20 z^{3} - 60 z^{2} + 120 z - 120\right) e^{z}`],
];
[a, ex] = params[pseudorandom % params.length];
```

Example card `Front` field:

```latex
\[
\int_{${a}}^{${b}} w^2 \ln w \,dw
\]
```

Similarly for `Back` field. Note `a` and `b` are interpolated when card is displayed.

In [169]:
import sympy as sy
from sympy import sympify as sfy
from IPython import display
from fractions import Fraction as R
import itertools, functools

In [3]:
a,b,c,d,e,s,t,u,v,w,x,y,z,theta = sy.symbols('a,b,c,d,e,s,t,u,v,w,x,y,z,theta')

In [233]:
def _generate_variable_value_pairs(variable, values):
    return ((variable, value) for value in values)

def _generate_variable_value_pair_lists(params):
    return (_generate_variable_value_pairs(variable, values) for (variable, values) in params.items())

def generate_args_lists(params):
    return itertools.product(*_generate_variable_value_pair_lists(params))

def _subs_one_item(ex, item):
    return ex.subs(*item)

def subs_items(ex, args_list):
    return functools.reduce(
        _subs_one_item,
        args_list,
        ex
    )

def extract_variables(args_list):
    return (var for (var, val) in args_list)

def extract_values(args_list):
    return (val for (var, val) in args_list)

def filter_args_lists(args_lists, filter_func = None):
    if filter_func is None:
        return args_lists
    return filter(lambda args_list: filter_func(*extract_values(args_list)), args_lists)

def default_get_probsol(ex, args_list):
    prob = subs_items(ex, args_list)
    sol = prob.doit()
    keep = True
    return keep, prob, sol

def get_params_script(
    ex,
    params,
    show_vars = True, show_prob = True, show_sol = True,
    probsol_func = None,
):
    if probsol_func is None:
        probsol_func = default_get_probsol
    if not (show_vars or show_prob or show_sol):
        raise Exception("at least one of show_vars or show_prob or show_sol should be True")
    script_lines = []
    _params = params.copy()
    variables = _params.keys()
    vars_str = f",{','.join(str(var) for var in params.keys())}" if show_vars else ""
    prob_var_str = ",prob" if show_prob else ""
    sol_var_str = ",sol" if show_sol else ""
    params_str = f"params{vars_str}{prob_var_str}{sol_var_str}"
    script_lines.append(f"if (typeof params === 'undefined') {{ let {params_str}; }}")
    script_lines.append("params = [")
    for args_list in generate_args_lists(_params):
        vals_str = f"{','.join(str(val) for val in extract_values(args_list))}," if show_vars else ""
        result = probsol_func(ex, args_list)
        keep, prob, sol = result[:3]
        if len(result) > 3:
            extra_params, extra_vals = result[3:]
            params_str = f"{params_str},{','.join(extra_params)}"
        else:
            extra_params = extra_vals = None
        if keep:
            prob_str = f"String.raw`{sy.latex(prob)}`," if show_prob else ""
            sol_str = f"String.raw`{sy.latex(sol)}`," if show_sol else ""
            extra_str = ','.join(str(x) for x in extra_params) if extra_params else ""
            script_lines.append(f"  [{vals_str}{prob_str}{sol_str}{extra_str}],")
    script_lines.append("];")
    script_lines.append("//pseudorandom=0;")
    script_lines.append(f"[{params_str.split(',',1)[-1]}] = params[pseudorandom % params.length];")
    return '\n'.join(script_lines)